# **D-download_dataset.ipynb**

Author: Zhixian Yang

Email: [yangzhx28@mail2.sysu.edu.cn](mailto:yangzhx28@mail2.sysu.edu.cn) or [yimu01439@gmail.com](mailto:yimu01439@gmail.com)

GitHub: [https://github.com/koar-create](https://github.com/koar-create)

Date created: July 20th, 2023

Last modified: July 26th, 2023

<br><br>

---

<br><br>

## **Description**
This document is a Jupyter Notebook designed for an exercise derived from the "Computational Tools for Climate Science 2023" course offered by Climatematch Academy. The code presented here comprises a combination of materials provided in the course and code obtained from online sources.

<br></br>
# **<font size=5 color='F00000'>download dependencies, import packages</font>**

In [ ]:
!pip install cdsapi --quiet

import cdsapi
import urllib.request
from itertools import product
import s3fs, boto3, botocore, pooch
from pythia_datasets import DATASETS
import os, sys, glob, platform, tempfile
from datetime import datetime, timedelta
import numpy as np, pandas as pd, xarray as xr
import matplotlib as mpl, matplotlib.pyplot as plt
import cartopy, cartopy.crs as ccrs, cartopy.feature as cfeature
from cartopy.mpl.ticker import LongitudeFormatter, LatitudeFormatter
import cartopy.io.shapereader as shapereader

<br></br>
# **<font size=5 color='F00000'>download NCEP-NCAR version 1 ReAnalysis Dataset</font>**

---

**site:**
- **https://downloads.psl.noaa.gov/Datasets/ncep.reanalysis**

**<font face="Times New Roman" size=4 color='43B0D0'>Yang: here I define a function used to download NCEP-NCAR version 1 ReAnalysis dataset. The new function is defined to make it easier to specify the time domain, spatial domain, and resolution we want. You don't need to understand this code block lines by lines.</font>**

In [ ]:
# Yang: Here I define a function used to download NCEP-NCAR version 1 ReAnalysis dataset. 
# You don't need to understand this code block lines by lines. 
def download_from_ncep(year=None, dataset='NCEP-NCAR-v1', 
                       temporal_res='monthly', level_type='pressure'):
    variables  = {'surface': ['lftx.sfc', 'lftx4.sfc', 'pres.sfc', 'pr_wtr.eatm', 'slp'], 
                  'near_surface': [i + '.sig995' for i in ['air', 'omega', 'pottmp', 'rhum', 'uwnd', 'vwnd']], 
                  'pressure': ['air', 'hgt', 'omega', 'rhum', 'shum', 'uwnd', 'vwnd']}
    extra_variables = {'surface': ['air', 'pres', 'pr_wtr', 'rhum', 'uwnd', 'vwnd', 'wspd', 
                                   'thickness', 'thickness_500200', 'thickness_850500', 'thickness_1000500'], 
                       'near_surface': ['wspd.sig995'], 
                       'pressure': ['wspd', 'pottmp']}
    if temporal_res == 'monthly':
        var_list = variables[level_type] + extra_variables[level_type]
    else:
        var_list = variables[level_type]
    for var in var_list:
        if   temporal_res == 'monthly':
            add_flag = 'Monthlies/'
            filename = f"{var}.mon.mean.nc"
        elif temporal_res == 'daily':
            add_flag = ''
            filename = f"{var}.{year}.nc"
        
        url = f"https://downloads.psl.noaa.gov/Datasets/ncep.reanalysis/{add_flag}{level_type}/{filename}"
        save_path = os.path.join(dataset, temporal_res, level_type)
        url = url.replace('near_', '') if level_type == 'near_surface' else url  # don't need to know what it does
        
        # if the path to save the file does not exist, create it first. 
        if not os.path.exists(save_path):
            os.makedirs(save_path)
        # if the file does not exist, download it. 
        if not os.path.exists(os.path.join(save_path, filename)):
            try:
                print(f'Downloading {url}...')
                urllib.request.urlretrieve(url, os.path.join(save_path, filename))
            except:
                print(f'{url} does not exist. ')
                continue

**<font face="Times New Roman" size=4 color='43B0D0'>Yang: ↓ this is the code block used to download data. specify the start year, end year, level type (`surface`, `near_surface` or `pressure`) and temporal resolution (`daily` or `monthly`).</font>**

In [ ]:
level_type = 'pressure'
temporal_res = 'monthly'
start_year, end_year = 1995, 2000

for year in range(start_year, end_year + 1):
    download_from_ncep(level_type=level_type, 
                       year=year, temporal_res=temporal_res, 
                       dataset='NCEP-NCAR-v1')

**<font face="Times New Roman" size=4 color='43B0D0'>Yang: ↓ these are the code blocks used to read the downloaded data. (you can try to unfold some of them)</font>**

In [ ]:
temporal_res = 'daily'
start_year, end_year = 1995, 2000
sfc_var_list  = ['lftx.sfc', 'lftx4.sfc', 'pres.sfc', 'pr_wtr.eatm', 'slp']

for idx, var in enumerate(sfc_var_list):
    ds_list = []
    for year in range(start_year, end_year + 1):
        file_path = os.path.join('NCEP-NCAR-v1', 'daily', 'surface', f"{var}.{year}.nc")
        ds_temp = xr.open_dataset(file_path)
        ds_list.append(ds_temp.sel(time=ds_temp.time.dt.hour == 0))
        del ds_temp
    ds_temp = xr.concat(ds_list, dim='time')
    ds_sfc_daily = ds_temp if idx == 0 else xr.merge([ds_sfc_daily, ds_temp])
    del ds_temp

In [ ]:
temporal_res = 'daily'
start_year, end_year = 1995, 2000
nsfc_var_list = [i + '.sig995' for i in ['air', 'omega', 'pottmp', 'rhum', 'uwnd', 'vwnd']]

for idx, var in enumerate(nsfc_var_list):
    ds_list = []
    for year in range(start_year, end_year + 1):
        file_path = os.path.join('NCEP-NCAR-v1', 'daily', 'near_surface', f"{var}.{year}.nc")
        ds_temp = xr.open_dataset(file_path)
        ds_list.append(ds_temp.sel(time=ds_temp.time.dt.hour == 0))
        del ds_temp
    ds_temp = xr.concat(ds_list, dim='time')
    ds_nsfc_daily = ds_temp if idx == 0 else xr.merge([ds_nsfc_daily, ds_temp])
    del ds_temp

In [ ]:
temporal_res = 'daily'
start_year, end_year = 1995, 2000
pres_var_list = ['air', 'hgt', 'omega', 'rhum', 'shum', 'uwnd', 'vwnd']

for idx, var in enumerate(pres_var_list):
    ds_list = []
    for year in range(start_year, end_year + 1):
        file_path = os.path.join('NCEP-NCAR-v1', 'daily', 'pressure', f"{var}.{year}.nc")
        ds_temp = xr.open_dataset(file_path)
        ds_list.append(ds_temp.sel(time=ds_temp.time.dt.hour == 0))
        del ds_temp
    ds_temp = xr.concat(ds_list, dim='time')
    ds_pres_daily = ds_temp if idx == 0 else xr.merge([ds_pres_daily, ds_temp])
    del ds_temp

In [ ]:
time = '2000-12-01'
level = 1000.0
var = ds3.omega.sel(time=time, level=level, lat=slice(10, -15), lon=slice(90, 130)).squeeze()

proj = ccrs.Mercator(central_longitude=110.0)
fig, ax = plt.subplots(subplot_kw={'projection': proj})
im = ax.contourf(var.lon, var.lat, var, transform=ccrs.PlateCarree(), 
                 levels=np.linspace(-0.4, 0.4, 11), extend='both', cmap='coolwarm')

gl = ax.gridlines(crs=ccrs.PlateCarree(), draw_labels=True, x_inline=False, y_inline=False, color='k', alpha=0.8, linewidth=0.8, linestyle='dashed')
gl.xformatter, gl.yformatter = LongitudeFormatter(), LatitudeFormatter()
gl.top_labels = False

ax.coastlines()
ax.set_title(f"{var.attrs['var_desc']}, {level:.0f} hPa, {time}")

fig.colorbar(im, label='units: Pa/s', orientation='horizontal', 
             ticks=np.linspace(-0.4, 0.4, 11), extend='both')

In [ ]:
temporal_res = 'monthly'
sfc_var_list  = ['lftx.sfc', 'lftx4.sfc', 'pres.sfc', 'pr_wtr.eatm', 'slp', 'air', 'pres', 'pr_wtr', 'rhum', 'uwnd', 'vwnd', 'wspd', 
                 'thickness', 'thickness_500200', 'thickness_850500', 'thickness_1000500']

for idx, var in enumerate(sfc_var_list):
    file_path = os.path.join('NCEP-NCAR-v1', 'monthly', 'surface', f"{var}.mon.mean.nc")
    ds_temp = xr.open_dataset(file_path)
    ds_sfc_monthly = ds_temp if idx == 0 else xr.merge([ds_sfc_monthly, ds_temp])
    del ds_temp

In [ ]:
temporal_res = 'monthly'
nsfc_var_list  = [i + '.sig995' for i in ['air', 'omega', 'pottmp', 'rhum', 'uwnd', 'vwnd', 'wspd']]

for idx, var in enumerate(nsfc_var_list):
    file_path = os.path.join('NCEP-NCAR-v1', 'monthly', 'near_surface', f"{var}.mon.mean.nc")
    ds_temp = xr.open_dataset(file_path)
    ds_nsfc_monthly = ds_temp if idx == 0 else xr.merge([ds_nsfc_monthly, ds_temp])
    del ds_temp

In [ ]:
temporal_res = 'monthly'
pres_var_list  = ['air', 'hgt', 'omega', 'rhum', 'shum', 'uwnd', 'vwnd', 'wspd', 'pottmp']

for idx, var in enumerate(pres_var_list):
    file_path = os.path.join('NCEP-NCAR-v1', 'monthly', 'surface', f"{var}.mon.mean.nc")
    ds_temp = xr.open_dataset(file_path)
    ds_pres_monthly = ds_temp if idx == 0 else xr.merge([ds_pres_monthly, ds_temp])
    del ds_temp

# **<font size=5 color='Magenta'>ERA5</font>**

---

**site:**
- **land-only, hourly: https://cds.climate.copernicus.eu/cdsapp#!/dataset/reanalysis-era5-land**
- **land-only, monthly: https://cds.climate.copernicus.eu/cdsapp#!/dataset/reanalysis-era5-land-monthly-means**
- **single level, hourly: https://cds.climate.copernicus.eu/cdsapp#!/dataset/reanalysis-era5-single-levels**
- **single level, monthly: https://cds.climate.copernicus.eu/cdsapp#!/dataset/reanalysis-era5-single-levels-monthly-means**
- **pressure levels, hourly: https://cds.climate.copernicus.eu/cdsapp#!/dataset/reanalysis-era5-pressure-levels**
- **pressure levels, monthly: https://cds.climate.copernicus.eu/cdsapp#!/dataset/reanalysis-era5-pressure-levels-monthly-means**

**<font face="Times New Roman" size=4 color='43D0B0'>Yang: To download ERA-5 ReAnalysis data offered by ECMWF (European Centre for Medium-Range Weather Forecasts), you need to register an account [here](https://cds.climate.copernicus.eu/user/register?destination=/), and obtain your private key [here](https://cds.climate.copernicus.eu/api-how-to). I hide my private key, so you're actually unable to run code blocks in this part. </font>**

**<font face="Times New Roman" size=4 color='43B0D0'>Yang: here I define a function used to download ERA-5 ReAnalysis dataset. Also, you don't need to understand this code block lines by lines.</font>**

In [ ]:
# 'S': ['10m_u_component_of_wind', '10m_v_component_of_wind', '100m_u_component_of_wind', '100m_v_component_of_wind', 
#                        'skin_temperature', '2m_temperature', 'sea_surface_temperature', 
#                        'surface_pressure', 'mean_sea_level_pressure', 
#                        'convective_precipitation', 'convective_rain_rate', 'large_scale_precipitation', 'large_scale_rain_rate', 'total_precipitation'],

def download_from_ecmwf(mode='LM', extra_flag="no", fname=None, 
                        years=None, months=None, days=None, times=None, 
                        area=[90, -180, -90, 180], pressures=None):
    !echo "url: https://cds.climate.copernicus.eu/api/v2" > $HOME/.cdsapirc
    !echo "key: ...:..." >> $HOME/.cdsapirc
    !cat $HOME/.cdsapirc
    temporal_res = {'H': 'hourly', 'M': 'monthly'}
    level_type   = {'L': 'land'  , 'S': 'single_level', 'P': 'pressure_levels'}
    file_name = os.path.join("ERA5", temporal_res[mode[-1]], level_type[mode[0]], fname)
    line1 = {'LH':            'land', 'LM':            'land-monthly-means',
             'SH':   'single-levels', 'SM':   'single-levels-monthly-means',
             'PH': 'pressure-levels', 'PM': 'pressure-levels-monthly-means'}
    product_type = {'H': 'reanalysis', 'M': 'monthly_averaged_reanalysis'}
    variables = {'L': ['10m_u_component_of_wind', '10m_v_component_of_wind', 
                       'skin_temperature', '2m_temperature', 
                       'surface_pressure', 
                       'total_precipitation'], 
                 'S': ['10m_u_component_of_wind', '10m_v_component_of_wind', 
                       'sea_surface_temperature', 
                       'mean_sea_level_pressure'],
                 'P': ['u_component_of_wind', 'v_component_of_wind', 'vertical_velocity', 
                        'temperature', 
                        'geopotential', 
                        'relative_humidity', 'specific_humidity']}
    extra_variables = {"yes": {'L': ['2m_dewpoint_temperature'],
                               'S': ['2m_dewpoint_temperature', 'convective_available_potential_energy', 
                                     'mean_vertically_integrated_moisture_divergence'],
                               'P': ['divergence', 'fraction_of_cloud_cover', 'ozone_mass_mixing_ratio', 
                                     'potential_vorticity', 'specific_cloud_ice_water_content', 
                                     'specific_cloud_liquid_water_content', 'specific_rain_water_content', 
                                     'specific_snow_water_content', 'vorticity', ], },
                       "no": {'L': [], 'S': [], 'P': [], }}
    pressure_level = {'pressure_level': pressures}
    dict_day = {'day': days}
    
    c = cdsapi.Client()
    retrieve_dict = {
        'product_type': product_type[mode[-1]],
        'variable': variables[mode[0]] + extra_variables[extra_flag][mode[0]],
        'year': years,
        'month': months,
        'time': times,
        'format': 'netcdf',
        'area': area}
    if mode == 'LH':
        retrieve_dict = {key: value for key, value in retrieve_dict.items() if key not in {'product_type': []}}
    if mode[0] == 'P':
        retrieve_dict.update(pressure_level)
    if mode[1] == 'H':
        retrieve_dict.update(dict_day)
    if not os.path.exists(os.path.dirname(file_name)):
        os.makedirs(os.path.dirname(file_name))
    if not os.path.exists(file_name):
        c.retrieve(f"reanalysis-era5-{line1[mode]}", retrieve_dict , file_name)

**<font size=4>Yang: ↓ this is the code block used to download data. specify the start year, end year, mode (`LH`, `LM`, `SH`, `SM`, `PH`, `PM`) and pressures (usually `['1000']` is ok).</font>**

In [ ]:
# specify the start year, end year, mode, and pressure level (if you choose 'pressure levels'). 
start_year, end_year = 1996, 2000
mode  = 'SM' # 'S' means 'single', 'M' means 'monthly', so there're {'L', 'S', 'P'}x{'H', 'M'} = 6 groups
pressures = ['1000']

# don't change these codes:
fname = f"{start_year}-{end_year}_{mode}.nc"
extra_flag = "no"
years = [str(year) for year in range(start_year, end_year + 1)]
months = [f"{month:02}" for month in range(1, 12 + 1)]
days = [f"{day:02}" for day in range(1, 31 + 1)]
times = ['00:00']
area = [60, -180, -60, 180]

# call the function. 
download_from_ecmwf(mode=mode, extra_flag="no", fname=fname, 
                    years=years, months=months, days=days, times=times, 
                    area=area, pressures=None)

**<font size=4>Yang: ↓ these are the code blocks used to read the downloaded data. </font>**

In [ ]:
file_path = os.path.join('ERA5', 'hourly', 'single_level', '1996-2005.SH.nc')
ds = xr.open_dataset(file_path).squeeze()
ds

**<font size=4>Yang: ↓ these are the code blocks used to visualize data. </font>**

In [ ]:
# time = '2000-12-01'
# level = 1000.0
var = ds4.lsrr.sel(latitude=slice(10, -15), longitude=slice(90, 130)).squeeze()

proj = ccrs.Mercator(central_longitude=110.0)
fig, ax = plt.subplots(subplot_kw={'projection': proj})
im = ax.contourf(var.longitude, var.latitude, 86400.0 * var, transform=ccrs.PlateCarree(), 
                 levels=np.linspace(0, 15, 11), extend='both', cmap='coolwarm')

gl = ax.gridlines(crs=ccrs.PlateCarree(), draw_labels=True, x_inline=False, y_inline=False, color='k', alpha=0.8, linewidth=0.8, linestyle='dashed')
gl.xformatter, gl.yformatter = LongitudeFormatter(), LatitudeFormatter()
gl.top_labels = False
gl.right_labels = False

ax.coastlines()
ax.set_title(f"{var.attrs['long_name']}, 2016-01-01")

fig.colorbar(im, label='units: m/day', orientation='horizontal', 
             ticks=np.linspace(0, 15, 11), extend='both')

# **<font size=5 color='cyan'>TRMM</font>**

---

**site:**
- **Register here first: https://registration.pps.eosdis.nasa.gov/registration/newContact.html**
- **After registration, you can access data here with username and password (both are the email you use to register: https://arthurhouhttps.pps.eosdis.nasa.gov/**

**<font size=4>Yang: You don't need to understand this code block, and don't try to run it (to successfully run it, you need to make some adaptations).</font>**

In [ ]:
import urllib.request
import os, time

username = '...'
password = '...'

start_year, end_year = 2001, 2002
for year in range(start_year, end_year + 1):
    version = '7' if ((year < 2000)|(year > 2010)) else '7A'
    for month in range(1, 12 + 1):
        for day in range(1, 31 + 1):
            for hour in range(0, 3, 3):
                url = f"https://arthurhouhttps.pps.eosdis.nasa.gov/trmmdata/ByDate/V07/{year}/{month:02}/{day:02}/3B42.{year}{month:02}{day:02}.{hour:02}.7A.HDF5"
                save_path = os.path.join('TRMM', '3B42', f"{year}", f"{month:02}", f"{day:02}", f"3B42.{year}{month:02}{day:02}.{hour:02}.7A.HDF5")
                
                # Create a password manager and add the username and password
                password_mgr = urllib.request.HTTPPasswordMgrWithDefaultRealm()
                password_mgr.add_password(None, url, username, password)

                # Create a basic authentication handler
                auth_handler = urllib.request.HTTPBasicAuthHandler(password_mgr)

                # Create an opener and install the authentication handler
                opener = urllib.request.build_opener(auth_handler)

                # Install the opener to be used by urlretrieve
                urllib.request.install_opener(opener)

                # Download the file using urlretrieve
                if not os.path.exists(os.path.dirname(save_path)):
                    os.makedirs(os.path.dirname(save_path))
                if not os.path.exists(save_path):
                    try:
                        urllib.request.urlretrieve(url, filename=save_path)
                        time.sleep(0.1)
                    except:
                        print(f"{save_path.split('/')[-1]} doesn't not exist. ")